# Intro

## Statistical and Dynamical Downscaling
Global Climate Models, also referred to as GCMs, are incredibly valuable tools in that they can capture large scale climate dynamics (e.g. future greenhouse gas concentration over the entire Earth).  Furthermore, many of these phenomena must be studied at a global scale.  However, there are limitations to GCMs.  Primarily, given the large scale analysis, grid cells and computation points for these models are actually quite large.  This leads to problems when trying to study local phenomena, such as the climate of a specific city or region.  This is when downscaling must be used.  There are two types of downscaling: Dynamical and Statistical. 

### Dynamical Downscaling
This type of downscaling uses a Regional Climate Model (RCM) which takes in the output of a GCM as a boundary condition.  Then, RCM use these boundary conditions and physical principles to generate higher resolution local climate.  However, thse are computationally expensive.

### Statistical Downscaling
Statistical Downscaling is another common method dependent on the availability of local weather data, such as precipitation.  A statistical relationship is developed between the historic observed climate data and the output of the climate model for the same historical period.  Then, this statistical relationship is used to downscale the climate model output to the local scale.  This can be combined with bias correction and adjustment to improve the accuracy of the downscaled data.

For this project, we will be using statistical downscaling in that we will attempt to determine a relationship between global predictors like temperature and pressure and local precipitation.  Then, we will use this relationship to downscale the precipitation data.

### Statistical Downscaling Methods
Statistical Downscaling, SD, consists of a identified historical period (calibration) used to analyze the predictor/predictand relationship.  This relationship is then applied to predictors from the GCMs to get the local climate projections. 

:::{.callout-note}
Sources include the Copernicus Climate Change Service (C3S), National Oceanic and Atmospheric Administration (NOAA), Evaluaation of statistical downscaling methods for climate change projections over Spain: Present conditions with perfect predictors (2021).
:::

# Project
For this project, I will be conducting a slightly different analysis than others in the class.  As I proposed to Dr. Doss-Gollin, I want to undertake a deep dive into a Julia based Downscaling Package, RainFARM, analyze the methods and algorithms used, and apply it to the larger scale precipitation data.

# RainFARM
According to Rebora et al. (2006b), RainFARM aims to generate "an ensemble of stochastic realizations of the small-scale precipitation fields that have statistical properties similar to those measured for rainfall in a given area and/or synoptic situation."  In other words, RainFARM takes course data and downscales it to a finer scale while still keeping similar statistical properties (e.g. it presents a faithful representataion of what your data could look like on the finer scale).  However, it is important to note that it is a stochastic process, and randomness is involved.  Consequently, it is not a deterministic process, and the results will vary each time the code is run.  This is why it is important to run the code multiple times to see many different possible downscaled outcomes.

## Common Rainfall Downscaling Methods Overview
There are many different methods used for downscaling, but there are three main types used in the literature, according to the Rebora article.  These are:
1. Point Processes
2. Autoregressive Processes with Nonlinear Transformation: This involves using patterns from the past (autoregressive) and applying it to a static nonlinear transformation.
3. Fractal Cascades: This creates a model where rain repeats in patterns inspired by fractals.
However, there have been examples of people combining these methods to create hybrid models.

## Methodology Outline
The RainFARM methodology uses a two step process:
1. Linear Autoregressive Process: The behavior of the rainfall on the course scale can be analyzed for patterns over this large area.  A simple forecast is made from this step.
2. Nonlinear Filtering: This is applied to the baseline prediction, and it adds detail and makes adjustments to increase accuracy.
This methodology works well with overall weather, and it uniquely takes into account the energy distribution of big-scale rain events.

## Inputs, Outputs, and Variables

### Inputs
RainFARM can input a course rainfall field P(X, Y, T).  This type of data can come from a few different sources, like GCMs, but it lacks the resolution at a fine scale needed for local analysis and regional studies.  X and Y represent the latitude and longitude of the grid on which data is stored, and T indicates time.  

$$
P(X, Y, T) = Input
L_{0} <= (X,Y) <= L_{max}
T_{0} <= T <= T_{max}
$$

The second two equations above define the range of scales for L and T.  These ranges refer to the scale of the data.  L0 and T0 are defined at the scales on which the data forecast becomes poor.  Typically, the spatial forecase becomes poor around 10-20 km, and the temporal forecast becomes poor around 2-4 hours for the types of data on which RainFARM is used.

### Outputs

The goal of the RainFARM package is to output a new precipitation field, written as r(x,y,t).  The new scale, lambda, will be less than the original scale, L, and typically falls around  1-2 km.  The new time scale, tau, will be less than the original time scale, T, and typically falls around 10 minutes.

This output must be thought of as one potential realization of the small scale behavior of this larger system and phenomena.

$$
r(x, y, t) = output 
\lambda_{0} < L_{0}
\tau_{0} < T_{0}
$$

### Aggregation and Validation

Interestingly, RainFARM includes its own self validation in which the output, the fine scale precipitation field, r(x,y,t), is aggregated on the original spatial and temporal scales.  This aggregated field should produce a coarse scale precipitation field, R(X, Y, T) that must be equal to the original field P.

## Detailed Methodology
The paper first describes the estimation of the space-time power spectrum of the field P.  This space-time power spectrum can be thought of as the energy distribution of the field P.  It helps describe how precipitation is generated across space and time, and is frequency used in meteorology.

$$
|\hat{P}(K_{X}, K_{Y},\Omega|^2 = space \space time \space power \space spectrum
$$
$$
K_{X} and K_{Y} = wavenumbers \space in \space X \space and \space Y \space directions
$$
$$
\Omega = angular \space frequency
$$
$$
(K_{X}, K_{Y}) <= pi/L_{0} = Nyquist \space Number
$$
$$
\Omega <= pi/T_{0} = Nyquest \space Frequency
$$

One of the first discussions of this new atmospheric energy balance methodology is in Equations Governing the Energetics of the Larger Scales of Atmospheric Turbulence in the Domain of Wave Number written by Barry Saltsman in 1957.  This paper took the previous understanding of turbulence and atmospheric energy, based heavily on Fourier analysis, and used to extend the "Reynolds formulation of the turbulence problem into the domain of scale."  Importantly, "such an approach makes it possible to investigate the energetics of the atmospheric disturbnces directly from daily observational data."  In other words, a precipitation field P can be analyzed with this space time power spectrum analysis to determine the energy dynamics of the atmosphere.

The wavenumbers allow for a representation of the spatial frequency of these energy waves.  K(X) and K(Y) represent how these precipitation patterns (and energy) vary in the X and Y directions.  However, the Nyquist Number defines the limitations of our understanding, but an in-depth discussion of this determination is outside the scope of this work.

The angular frequency, in the general theory, measures how fast the waves oscillate in time.  In other words, it represents how fast the pattern changes, so for RainFARM it represents how quickly the precipitation patterns change.  Again, the Nyquist Frequency defiiines the limitations of the analysis.

Next, the power spectrum is extrapolated to small scales.  Rebora describes that a power-law behavior is assumed; in other words, the the power relationships of the coarse data are assumped proportional to the power relationships of the fine data.  This allows for a relationship between the coarse and fine data to be determined.  In addition, the logarithmic slopes \alpha and \beta represent the spatial slope and the temporal slope, respectively.  These help determine the rate at which the energy/precipitation changes based on these variables.  In other words, we define a rate of change.

The model assumpes isotropy, meaning that the precipitation field will behave uniformly in all spatial directions.  This uniform behavior in all directions lessens the computational load and simplifies the model.

A Fourier spectrum is defined:

$$
\hat{g}(k_{x},k_{y},\omega) = |\hat{g}(k_{x}, k{y}, \omega)|e^{i\phi}
$$

This Fourier analysis lends itself to an exploration of the frequency components of the data and a consequent modeling of the distribution of power across frequencies.  An inversion creates a Gaussian field that represents the original data in space time.  The more-specific steps are listed below:

1. Fourier Spectrum: This breaks our power spectrum into its frequency components.  A functional form becomes the equation shown below.  Notice it is dependent on the wavenumbers and the angular frequency, discussed earlier.

$$
|\hat{g}(k_{x}, k{y}, \omega)|^2 \propto (k_{x}^2 + k_{y}^2)^{-\alpha/2} \omega^{-\beta}
$$

2. Form for Power Spectrum: This allows us to describe the power spectrum in a mathematical form.  This gives predictive power.

3. Inversion: The frequency domain, with our predictive power given the equation governing power spectra, can be inverted to generate a Gaussian field that represents the original data in space time.

4. Normalization

At this point in the process, (1) a coarse data field has been taken in, (2) a power spectrum has been generated to represent the energy/precipitation in the atmosphere, (3)  the power spectrum is extrapolated to smaller scales with the assumption of power-law behavior, (4) fourier spectrum are used to create a function that represents this downscaled power spectrum and then inverted to create a gaussian field that brings the downscaled data back to a space time field.

Next, a precipitation field is generated by taking "a nonlinear transformation of the Gaussian field g" (Rebora et al., 2006b).  The transformation is defind as:

$$
\tilde{r}(x,y,t) = exp[g(x,y,t)]
$$


Notice that this transformation is an exponential function applied to our Gaussian field (e.g. the space time field of the downscaled data), so it creates a lognormal field.  This is a nonlinear transformation, and it is applied to the Gaussian field to create the precipitation field.  

As mentioned earlier, there is an internal validation step in which the generated (or synthetic) precipitation field is compared to the original field.  This is done by aggregating the synthetic field to the original scale; in other words, the small scale field is aggregated to L0 and T0.

In each box, the ratio of the original field, P, and the synthetic field, R, is calculated.  If the ratio is 1, there is a perfect match.  If it is not one, the synthetic field R can be adjusted.  This means that the synthetic field closely resembles the original field at the L0 and T0 scale.

## Important Notes
As discussed, the RainFARM methodology is stochastic, and this random behavior is associated with the Fourier phases. The phases are randomly generated, and this randomness is what creates the different realizations of the small scale precipitation field.  This is why it is important to run the code multiple times to see many different possible downscaled outcomes.  The original paper introducing this method writes: "The RainFARM approach is easily used for ensemble predictions; by choosing different sets of random Fourier phases, one can generate a large number of stochastic fields that are all equal to P when aggregated on space and time scales larger than L0 and To, and are different, but with similar statistical properties, on smaller scales."

# Packages


In [ ]:
using Plots
using StatsBase
using StatsPlots
using Distributions
using Statistics
using CDSAPI
using NCDatasets
using StatsBase: shuffle
using NCDatasets
using RainFARM
using DataFrames

# Downloading Data


In [ ]:
precip_tx = NCDataset("data/precip_tx.nc")
temp_2015 = NCDataset("data/raw/2m_temperature_2015.nc")
temp_2016 = NCDataset("data/raw/2m_temperature_2016.nc")
temp_2017 = NCDataset("data/raw/2m_temperature_2017.nc")
temp_2018 = NCDataset("data/raw/2m_temperature_2018.nc")
temp_2019 = NCDataset("data/raw/2m_temperature_2019.nc")
temp_2020 = NCDataset("data/raw/2m_temperature_2020.nc")
press_2015 = NCDataset("data/raw/500hPa_geopotential_2015.nc")
press_2016 = NCDataset("data/raw/500hPa_geopotential_2016.nc")
press_2017 = NCDataset("data/raw/500hPa_geopotential_2017.nc")
press_2018 = NCDataset("data/raw/500hPa_geopotential_2018.nc")
press_2019 = NCDataset("data/raw/500hPa_geopotential_2019.nc")
press_2020 = NCDataset("data/raw/500hPa_geopotential_2020.nc")

display(precip_tx)

# Data Preprocessing


In [ ]:
precip_tx_p = precip_tx["precip"]

In [ ]:
# combine all temp files so there is a continuous time variable
# select the first 8760 hours of the 2016 data t2m
# 2016 is a leap year so it has 8784 hours
temp_2016 = temp_2016[:, :, 1:8760];
# create an empty matrix with dimensions 66, 27, 34680
temp_2015to2018                    = zeros(66, 27, 34680)
temp_2015to2018[:, :, 1:8760]      = temp_2015
temp_2015to2018[:, :, 8761:17340]  = temp_2016
temp_2015to2018[:, :, 17341:26010] = temp_2017
temp_2015to2018[:, :, 26011:34680] = temp_2018
temp15to18                         = temp_2015to2018

Now inputs to the RainFARM function must be determined, and they are informed by information from the methodology.

The first input into the function is the coarse precipitation data, or the coarse data P(X, Y, T).  In this case, the coarse data is the precipitation over Texas.

Te second input is the slope.  The slope is the spatial spectral slope.  This value helps quantify the rate at which the power in the precipitation field decreases as you move to small scales in space.


In [ ]:
using RainFARM
using DataFrames
default_value = 0.0
r = coalesce.(precip_tx_p, default_value)
slope = 1;
nf = 5;
downscaled_precip_tx = rainfarm(r, slope, nf)

This created a variable, downscaled_precip_tx, with the downscaled precipitation data over the same time period of the original precipitation data.

First, it is possible to analyze the original, coarse data and the downscaled data side by side.  This can be done by plotting the data at a specific time frame.  Consider 10 randomly selected precipitation fields.


In [ ]:
# plot one frame of the downscaled_precip_tx
# create a vector of 10 random numbers between 1 and 8760
# this will be used to select 10 random time frames to plot
tenrandom = rand(1:8760, 10)
for i in tenrandom
    timeframe = i
    downheatmap = heatmap(downscaled_precip_tx[:, :, timeframe])
    origheatmap = heatmap(precip_tx_p[:, :, timeframe])
    cumplot = plot(downheatmap, origheatmap, layout = (1, 2), legend = true)
    display(cumplot)
end

Visually, these side-by-side comparisons show reasonable results for the downscaling.  The highest precipitation hot spots are located in the same places across the coarse and fine data, and general spatial patterns are similar.

However, it is also important to remember, still, that this is a stochastic process.  Let's explore the different options that can be generated by this same function.

The code below will generate a random time step and then run the same function on the same time step.  The two outputs will be plotted side by side to demonstrate that there are variations within these results.


In [ ]:
using RainFARM
using DataFrames
default_value = 0.0
randomtimestep = rand(1:8760)
stoch_precip_tx_p = precip_tx_p[:, :, randomtimestep]
r = coalesce.(stoch_precip_tx_p, default_value)
slope = 1;
nf = 5;
downscaled_stoch1_precip_tx = rainfarm(r, slope, nf)
downscaled_stoch2_precip_tx = rainfarm(r, slope, nf)
stoch1_hm = heatmap(downscaled_stoch1_precip_tx[:, :, 1])
stoch2_hm = heatmap(downscaled_stoch2_precip_tx[:, :, 1])
cumplot = plot(stoch1_hm, stoch2_hm, layout = (1, 2), legend = true)

# Quantile Quantile Mapping
Quantile-Quantile Mapping, or Q-Q Mapping, can be used to downscale climate data. This method is used to adjust the distribution of a variable to match the distribution of another variable. In this case, we are using Q-Q Mapping to adjust the distribution of the precipitation data to match the observed temperature data.  Temperature data tends to be smoother and more reliable than precipitation data, so we are using the temperature data as a reference to adjust the precipitation data.

## Methodology
The Q-Q Mapping methodology is as follows:
1. Calculate the cumulative distribution function (CDF) of the observed temperature data and the CDF of the precipitation data.
    We know that the CDF returns the probabilities of X being less than or equal to some value x.
    $$
    Pr(X \leq x) = F(x)
    $$
2. Calculate the inverse of the CDF of the observed temperature data and the inverse of the CDF of the precipitation data.
    The inverse CDF can also be called the quantile function, and it can take this CDF and give us values for the quantiles we want!  It can be represented with the following equation:
    $$
    F^{-1}(p) = x
    $$
    It can also be conceptualized as choosing a value you want on the y axis (e.g. the 0.9 quantile or 90% probability of being less than the value x), tracing over to the graph until you hit the CDF, and then moving down to the x axis to find your value of x that corresponds to the 0.9 quantile.
3. Use the inverse of the CDF of the observed temperature data to transform the precipitation data to match the observed temperature data.


## Implementation
The Q-Q Mapping methodology was implemented in Julia using the following code:

In [ ]:
precipitation = [2,6,2,3,4,5,8,5,6,6,7,13,9,12,9,10,10,10,11,11,11,12,12,12,13]
temperature = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25]
function qqmap(precipitation::AbstractVector{<:Real}, temperature::AbstractVector{<:Real})
    cdf_precipitation = ecdf(precipitation)
    cdf_temperature = ecdf(temperature)
    adjusted_precip_data = [quantile(temperature, cdf_precipitation(x)) for x in precipitation]
    return adjusted_precip_data
end
newprecipitation = qqmap(precipitation, temperature)
compplot = plot(precipitation)
compplot = plot!(newprecipitation)

# Lab 06: PCA and kNN


## Methodology
The Lab 06 methodology is as follows:
1. Calculate the principal components of the observed temperature data.
2. Utilize the k-Nearest Neighbors algorithm to find the k nearest neighbors of the observed temperature data.
3. Use the k nearest neighbors of the observed temperature data to predict the precipitation data.

## Implementation
The Lab 06 methodology was implemented in Julia using the following code:


In [ ]:
# Assuming `precip` is your precipitation data variable
# Dimensions: lon × lat × time

# Create an empty array to store data without missing values
cleaned_precip_tx = Float32[]
# Loop through the time dimension
for t = 1:8760
    # Extract a 2D slice at time t
    slice = precip_tx[:, :, t]
    # Filter out missing values from the 2D slice
    #cleaned_slice = filter(x -> !ismissing(x), slice)
    # Append the cleaned slice to the cleaned_precip array
    #append!(cleaned_precip_tx, cleaned_slice)
end

# Calculate the new dimensions after removing missing values
#new_lon, new_lat, new_time = size(precip_tx)

# Reshape the cleaned data to match the original dimensions
#cleaned_precip_tx = reshape(cleaned_precip_tx, new_lon, new_lat, new_time)

# Now, `cleaned_precip` contains your data with missing values removed